In [7]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects import r as R
import rpy2.rinterface
pandas2ri.activate()

In [8]:
twin_file = "/ifs/projects/proj052/pipeline_proj052/twin_files.dir/P3-fano_CD4_Tmem-DZ.tsv"
twin_df = pd.read_table(twin_file, sep="\t", header=0, index_col=None)

In [9]:
marker_groups = twin_df.groupby(['marker'])

In [10]:
# set up rpy functions for linear model fit
r_lm = R["lm"]

In [25]:
frame_list = []
for name, group in marker_groups:
    group_df = group.copy()
    # convert to rpy2 dataframe
    print group_df.head()
    break
    r_group = pandas2ri.py2ri_pandasdataframe(group_df)
    
    # run linear model, adjusting for twin age and flow processing subset
    twin1_r = r_lm("twin1 ~ subset + age_x", data=r_group)
    twin2_r = r_lm("twin2 ~ subset + age_y", data=r_group)
    # pull out residuals
    twin1_residuals = twin1_r.rx("residuals")[0]
    twin2_residuals = twin2_r.rx("residuals")[0]
    
    # pull out indices and convert back to python objects, make sure index are integers
    tw1_indx = [int(ix1) for ix1 in pandas2ri.ri2py_listvector(twin1_residuals.dimnames)[0]]
    tw2_indx = [int(ix2) for ix2 in pandas2ri.ri2py_listvector(twin1_residuals.dimnames)[0]]
    
    tw1_rs = pandas2ri.ri2py_listvector(twin1_residuals)
    tw2_rs = pandas2ri.ri2py_listvector(twin2_residuals)

    tw1_ser = pd.Series(tw1_rs, index=tw1_indx, dtype=np.float64)
    tw2_ser = pd.Series(tw2_rs, index=tw2_indx, dtype=np.float64)

    res_df = pd.DataFrame([tw1_ser, tw2_ser]).T
    res_df.columns = ["twin1_res", "twin2_res"]

    # merge dataframe of residual and original dataframe
    m_df = pd.merge(left=group_df, right= res_df, left_index=True, right_index=True, how='inner')
    
    frame_list.append(m_df)

      indx     twin.id_x  twin_num_x  family_id  flowjo_id_x  age_x subset  \
1176  1176  Twin_TS01_10          10         10          356  64.45   ZZEV   
1177  1177  Twin_TS01_13          13         13          344  52.14   ZZEV   
1178  1178  Twin_TS01_14          14         14          208  72.04   ZZEV   
1179  1179  Twin_TS01_15          15         15            4  66.63   ZZEV   
1180  1180  Twin_TS01_16          16         16          202  71.80   ZZEV   

     zygosity  replicate_x  visit_x marker     twin1     twin.id_y  \
1176       DZ            1        1     Aq  0.069487  Twin_TS01_10   
1177       DZ            1        1     Aq  0.042880  Twin_TS01_13   
1178       DZ            1        1     Aq  0.851619  Twin_TS01_14   
1179       DZ            1        1     Aq  0.082516  Twin_TS01_15   
1180       DZ            1        1     Aq  0.050089  Twin_TS01_16   

      twin_num_y  flowjo_id_y  age_y  replicate_y  visit_y     twin2  
1176          10          355  64.45   

In [19]:
df = frame_list[0]
frame_list.remove(df)

for each in frame_list:
    _df = each
    df = df.append(_df)

In [23]:
df.head()

,indx,twin.id_x,twin_num_x,family_id,flowjo_id_x,age_x,subset,zygosity,replicate_x,visit_x,...,twin1,twin.id_y,twin_num_y,flowjo_id_y,age_y,replicate_y,visit_y,twin2,twin1_res,twin2_res
1680,1680,Twin_TS01_10,10,10,356,64.45,ZZEV,DZ,1,1,...,0.639506,Twin_TS01_10,10,355,64.45,1,1,0.639506,0.247687,0.247687
1681,1681,Twin_TS01_13,13,13,344,52.14,ZZEV,DZ,1,1,...,0.220005,Twin_TS01_13,13,343,52.14,1,1,0.220005,-0.196685,-0.196685
1682,1682,Twin_TS01_14,14,14,208,72.04,ZZEV,DZ,1,1,...,0.568242,Twin_TS01_14,14,207,72.04,1,1,0.568242,0.191760,0.191760
1683,1683,Twin_TS01_15,15,15,4,66.63,ZZEV,DZ,1,1,...,0.327038,Twin_TS01_15,15,3,66.63,1,1,0.327038,-0.060375,-0.060375
1684,1684,Twin_TS01_16,16,16,202,71.80,ZZEV,DZ,1,1,...,0.410739,Twin_TS01_16,16,201,71.80,1,1,0.410739,0.033771,0.033771


In [24]:
twin_df.head()

,indx,twin.id_x,twin_num_x,family_id,flowjo_id_x,age_x,subset,zygosity,replicate_x,visit_x,marker,twin1,twin.id_y,twin_num_y,flowjo_id_y,age_y,replicate_y,visit_y,twin2
0,0,Twin_TS01_10,10,10,356,64.45,ZZEV,DZ,1,1,Unnamed: 0,3,Twin_TS01_10,10,355,64.45,1,1,2
1,1,Twin_TS01_13,13,13,344,52.14,ZZEV,DZ,1,1,Unnamed: 0,9,Twin_TS01_13,13,343,52.14,1,1,8
2,2,Twin_TS01_14,14,14,208,72.04,ZZEV,DZ,1,1,Unnamed: 0,11,Twin_TS01_14,14,207,72.04,1,1,10
3,3,Twin_TS01_15,15,15,4,66.63,ZZEV,DZ,1,1,Unnamed: 0,13,Twin_TS01_15,15,3,66.63,1,1,12
4,4,Twin_TS01_16,16,16,202,71.80,ZZEV,DZ,1,1,Unnamed: 0,15,Twin_TS01_16,16,201,71.80,1,1,14
